<a href="https://colab.research.google.com/github/georgsvit/TT-18-ML/blob/main/InClass_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [18]:
train = pd.read_csv("/content/drive/MyDrive/InClass Data/train.csv")
train.rename(columns={train.columns[0]: "Id"}, inplace=True)
# train

In [19]:
test = pd.read_csv("/content/drive/MyDrive/InClass Data/test_no_target.csv")
test.rename(columns={test.columns[0]: "Id"}, inplace=True)
# test

In [20]:
# Data preparation

In [21]:
models_list = list(train['model'].unique())
onehotencoder = OneHotEncoder(handle_unknown='ignore')
onehotencoder.fit(np.array(models_list).reshape(-1,1))

def replace_values(data, feature_name):
  models = dict().fromkeys(data['model'].unique())

  for model in models.keys():
    temp = data[(data['model'] == model) & (pd.notnull(data[feature_name]))][feature_name].head(1)  
    if not temp.empty:
      models[model] = temp.iloc[0]
      
  for i, row in data.iterrows():
    if (pd.isna(row[feature_name])):
      data.at[i, feature_name] = models[row['model']] if not pd.isna(models[row['model']]) else models['andere']
  return data

def prepare_data(data):
  # Fix years
  data["registration_year"] = data["registration_year"].apply(lambda x: x if x >= 100 else x + 2000 if x < 22 else x + 1900)
  # Fix model
  data["model"].fillna("andere", inplace=True)
  # Fix gearbox
  data["gearbox"].fillna("undefined", inplace=True)
  # Fix type
  data["type"].fillna("other", inplace=True)
  # Fix fuel
  data["fuel"].fillna("other", inplace=True)

  # -------------------------------------------

  # Fix damage
  data["damage"].fillna(0.5, inplace=True)
  # Fix engine capacity
  data = replace_values(data, 'engine_capacity')
  # Fix
  data = replace_values(data, 'insurance_price')

  # MinMax
  scaler = MinMaxScaler()

  data[['engine_capacity']] = scaler.fit_transform(data[['engine_capacity']])
  data[['registration_year']] = scaler.fit_transform(data[['registration_year']])
  data[['power']] = scaler.fit_transform(data[['power']])
  data[['mileage']] = scaler.fit_transform(data[['mileage']])
  data[['insurance_price']] = scaler.fit_transform(data[['insurance_price']])

  # OneHot
  data = pd.get_dummies(data, columns=['type', 'gearbox', 'brand', 'fuel'])
  data.drop(['model'], axis=1, inplace=True)
  
  # res = onehotencoder.transform(np.array(data['model']).reshape(-1,1)).toarray()
  # df = pd.DataFrame(
  #     res,
  #     columns=[x for x in onehotencoder.get_feature_names_out(['model'])]
  # )
  
  # data = pd.concat([data, df], axis=1)
  # data = data.drop(['model'], axis=1)  

  return data

In [22]:
train = prepare_data(train)
test = prepare_data(test)
train

,Id,engine_capacity,registration_year,power,mileage,damage,zipcode,insurance_price,price,type_bus,type_convertible,type_coupé,type_limousine,type_other,type_small car,type_station wagon,gearbox_auto,gearbox_manual,gearbox_undefined,brand_alfa_romeo,brand_audi,brand_bmw,brand_chevrolet,brand_chrysler,brand_citroen,brand_dacia,brand_daewoo,brand_daihatsu,brand_fiat,brand_ford,brand_honda,brand_hyundai,brand_jaguar,brand_jeep,brand_kia,brand_lada,brand_lancia,brand_land_rover,brand_mazda,brand_mercedes_benz,brand_mini,brand_mitsubishi,brand_nissan,brand_opel,brand_peugeot,brand_porsche,brand_renault,brand_rover,brand_saab,brand_seat,brand_skoda,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,fuel_compressed natural gas,fuel_diesel,fuel_gasoline,fuel_liquefied petroleum gas,fuel_other
0,48298,0.210526,0.859155,0.008583,1.000000,0.0,49191,0.009499,4267,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,81047,0.231579,1.000000,0.000000,1.000000,0.5,45896,0.025931,2457,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,92754,0.231579,0.915493,0.010729,0.827586,0.0,59229,0.023620,10374,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,46007,0.242105,0.774648,0.016247,1.000000,0.0,39365,0.017202,7098,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,76981,0.168421,0.816901,0.006683,1.000000,0.0,55271,0.010526,2365,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,50429,0.147368,0.859155,0.004598,0.586207,0.0,35745,0.012580,4686,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
49996,64425,0.136842,0.830986,0.003678,1.000000,0.0,60386,0.005135,864,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
49997,90761,0.126316,0.718310,0.009196,1.000000,0.0,28309,0.003081,2275,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
49998,39709,0.315789,0.873239,0.007480,0.655172,0.0,83623,0.012580,8144,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [23]:
test

,Id,engine_capacity,registration_year,power,mileage,damage,zipcode,insurance_price,type_bus,type_convertible,type_coupé,type_limousine,type_other,type_small car,type_station wagon,gearbox_auto,gearbox_manual,gearbox_undefined,brand_alfa_romeo,brand_audi,brand_bmw,brand_chevrolet,brand_chrysler,brand_citroen,brand_dacia,brand_daewoo,brand_daihatsu,brand_fiat,brand_ford,brand_honda,brand_hyundai,brand_jaguar,brand_jeep,brand_kia,brand_lada,brand_lancia,brand_land_rover,brand_mazda,brand_mercedes_benz,brand_mini,brand_mitsubishi,brand_nissan,brand_opel,brand_peugeot,brand_porsche,brand_renault,brand_rover,brand_saab,brand_seat,brand_skoda,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,fuel_compressed natural gas,fuel_diesel,fuel_gasoline,fuel_liquefied petroleum gas,fuel_other
0,60314,0.164948,0.957746,0.008337,0.241379,0.0,30449,0.009234,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,12566,0.226804,0.830986,0.020414,1.000000,0.0,45307,0.012697,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,17760,0.206186,0.859155,0.010422,1.000000,0.0,59494,0.008657,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,8876,0.185567,0.760563,0.006192,1.000000,0.5,25524,0.000770,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,80392,0.226804,0.422535,0.003310,1.000000,0.0,70794,0.021162,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,93878,0.144330,0.760563,0.005272,1.000000,0.0,44339,0.001924,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
49996,99783,0.329897,0.802817,0.011280,1.000000,0.0,47574,0.004425,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
49997,57399,0.123711,0.845070,0.003188,0.655172,0.0,50389,0.000962,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
49998,97106,0.257732,0.788732,0.009257,1.000000,0.0,12209,0.017699,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0


In [24]:
# ----------------------------------------------------------------------------------

In [25]:
X, y = train.drop(columns=['Id','price']), train[['Id','price']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

test, id_test = test.drop(columns=['Id']), test[['Id']]
y_train = y_train.drop(columns=['Id'])

print('Train size: {}, CV size: {}, Test size: {}' .format(X_train.shape, X_test.shape, test.shape))

Train size: (45000, 62), CV size: (5000, 62), Test size: (50000, 62)


In [26]:
from math import sqrt
from scipy.stats import randint as sp_randint
from sklearn.metrics import mean_squared_error as mse
from scipy.stats import uniform
from lightgbm import LGBMRegressor, Dataset
from sklearn.model_selection import RandomizedSearchCV

In [27]:
lgb_model = LGBMRegressor(subsample=0.9)

params = {'learning_rate': uniform(0, 1),
          'n_estimators': sp_randint(200, 1500),
          'num_leaves': sp_randint(20, 200),
          'max_depth': sp_randint(2, 15),
          'min_child_weight': uniform(0, 2),
          'colsample_bytree': uniform(0, 1),
         }
lgb_random = RandomizedSearchCV(lgb_model, param_distributions=params, n_iter=10, cv=3, random_state=42, 
                                scoring='neg_root_mean_squared_error', verbose=10, return_train_score=True)
lgb_random = lgb_random.fit(X_train, y_train)

best_params = lgb_random.best_params_
print(best_params)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START colsample_bytree=0.3745401188473625, learning_rate=0.9507143064099162, max_depth=12, min_child_weight=1.5593820005455385, n_estimators=1244, num_leaves=122
[CV 1/3; 1/10] END colsample_bytree=0.3745401188473625, learning_rate=0.9507143064099162, max_depth=12, min_child_weight=1.5593820005455385, n_estimators=1244, num_leaves=122;, score=(train=-67.528, test=-3638.988) total time=   7.3s
[CV 2/3; 1/10] START colsample_bytree=0.3745401188473625, learning_rate=0.9507143064099162, max_depth=12, min_child_weight=1.5593820005455385, n_estimators=1244, num_leaves=122
[CV 2/3; 1/10] END colsample_bytree=0.3745401188473625, learning_rate=0.9507143064099162, max_depth=12, min_child_weight=1.5593820005455385, n_estimators=1244, num_leaves=122;, score=(train=-83.387, test=-3629.078) total time=   7.2s
[CV 3/3; 1/10] START colsample_bytree=0.3745401188473625, learning_rate=0.9507143064099162, max_depth=12, min_child_w

In [28]:
model = LGBMRegressor(**best_params, subsample=0.9, random_state=42, n_jobs=-1)
# model = LGBMRegressor(subsample=0.9, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

lgb_preds_tr = model.predict(X_train)
lgb_pred_cv = model.predict(X_test)
lgb_preds_te = model.predict(test)

In [29]:
submission = pd.DataFrame({
        "Id": id_test["Id"],
        "Predicted": lgb_preds_te
    })
submission

,Id,Predicted
0,60314,10230.178783
1,12566,9011.838815
2,17760,6725.268470
3,8876,1006.643949
4,80392,6984.069225
...,...,...
49995,93878,1302.566603
49996,99783,2662.713280
49997,57399,2274.627188
49998,97106,7951.692035


In [30]:
submission.to_csv('submission.csv', index=False)